# [550. Game Play Analysis IV](https://leetcode.com/problems/game-play-analysis-iv/description/?envType=study-plan-v2&envId=top-sql-50)

Table: Activity

<pre>+--------------+---------+
| Column Name  | Type    |
+--------------+---------+
| player_id    | int     |
| device_id    | int     |
| event_date   | date    |
| games_played | int     |
+--------------+---------+</pre>
(player_id, event_date) is the primary key (combination of columns with unique values) of this table.
This table shows the activity of players of some games.
Each row is a record of a player who logged in and played a number of games (possibly 0) before logging out on someday using some device.


Write a solution to report the fraction of players that logged in again on the day after the day they first logged in, rounded to 2 decimal places. In other words, you need to count the number of players that logged in for at least two consecutive days starting from their first login date, then divide that number by the total number of players.

The result format is in the following example.



Example 1:

Input:
Activity table:
<pre>+-----------+-----------+------------+--------------+
| player_id | device_id | event_date | games_played |
+-----------+-----------+------------+--------------+
| 1         | 2         | 2016-03-01 | 5            |
| 1         | 2         | 2016-03-02 | 6            |
| 2         | 3         | 2017-06-25 | 1            |
| 3         | 1         | 2016-03-02 | 0            |
| 3         | 4         | 2018-07-03 | 5            |
+-----------+-----------+------------+--------------+</pre>
Output:
<pre>+-----------+
| fraction  |
+-----------+
| 0.33      |
+-----------+</pre>
Explanation:
Only the player with id 1 logged back in after the first day he had logged in so the answer is 1/3 = 0.33

In [1]:
#pandas schema
import pandas as pd

data = [[1, 2, '2016-03-01', 5], [1, 2, '2016-03-02', 6], [2, 3, '2017-06-25', 1], [3, 1, '2016-03-02', 0],
        [3, 4, '2018-07-03', 5]]
activity = pd.DataFrame(data, columns=['player_id', 'device_id', 'event_date', 'games_played']).astype(
    {'player_id': 'Int64', 'device_id': 'Int64', 'event_date': 'datetime64[ns]', 'games_played': 'Int64'})

# to spark dataframe

from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

activity_df = spark.createDataFrame(activity)

In [2]:
activity

,player_id,device_id,event_date,games_played
0,1,2,2016-03-01,5
1,1,2,2016-03-02,6
2,2,3,2017-06-25,1
3,3,1,2016-03-02,0
4,3,4,2018-07-03,5


In [18]:
# solving in spark dataframe
c=activity_df.select('player_id').distinct().count()

activity_df.groupBy('player_id').agg(min('event_date').alias('MD'))\
    .join(activity_df,['player_id'],'inner')\
    .where(datediff('event_date','MD')==1)\
    .select(round(count('player_id')/c,2).alias('fraction'))\
    .show()

+--------+
|fraction|
+--------+
|    0.33|
+--------+



In [4]:
# solving in Spark SQL

activity_df.createOrReplaceTempView('activity')

spark.sql('''
with fd as(
        select player_id,min(event_date) as md
        from activity
        group by player_id
    )

  select round(count(player_id)/(select count(distinct player_id) from activity),2) as  fraction
    from activity a
    join fd using(player_id)
    where DATEDIFF(event_date,md)=1

''').show()

+--------+
|fraction|
+--------+
|    0.33|
+--------+



In [ ]:
spark.stop()